# Cuaderno 12: Flujos multiproducto 
# (Formulación alternativa)

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$
$\newcommand{\ZZ}{\mathbb{Z}}$
$\newcommand{\RR}{\mathbb{R}}$

Dados: 
* un grafo dirigido $D=(V,A)$; 
* un conjunto $K$ de productos; 
* un vector de capacidades $u \in \ZZ^{A}_+$  asociadas a los arcos de $D$; 
* un vector $c \in \ZZ^{A}$ de costos de transporte unitarios asociados a los arcos de $D$; y, 
* un nodo origen $s_k$, un nodo destino $t_k$ y una demanda $d_k$ asociados a cada producto $k \in K$.

El *problema de flujo multiproducto de costo mínimo* consiste en encontrar un flujo $x \in \RR^{A \times K}$  que satisfaga las siguientes condiciones:

* para cada producto $k \in K$, el flujo neto en $s_k$ (es decir, el flujo total de $k$ sobre los arcos que entran a $s_k$ menos el flujo total de $k$ sobre los arcos que salen de $s_k$) debe ser igual al negativo de la demanda $-d_k$ del producto, el flujo neto en $t_k$ debe ser igual a la demanda $d_k$, y el flujo neto en los demás nodos debe ser cero;  
* para cada arco $(i,j) \in A$, el flujo agregado de todos los productos sobre $(i,j)$ debe ser menor o igual a su capacidad $u_{ij}$;
* el costo total del flujo debe ser mínimo. Este costo se calcula al multiplicar el flujo total sobre cada arco $(i,j) \in A$ por su correspondiente costo unitario de transportación $c_{ij}$, y sumar estos valores sobre todos los arcos de la red.


Utilizando variables continuas no negativas $x_{ij}^k$ para representar el flujo del producto $k \in K$ sobre el arco $(i, j) \in A$, el problema de flujo multiproducto de costo mínimo puede formularse como el siguiente programa lineal:

\begin{align*}
\min &\sum_{k \in K} \sum_{(i,j) \in A} c_{ij} x_{ij}^k\\ 
& \mbox{s.r.}\\
&\sum_{(j, i) \in A} x_{ji}^k - \sum_{(i, j) \in A} x_{ij}^k = b_i^k, \quad \forall i \in V, k \in K\\
&\sum_{k \in K} x_{ij}^k \leq u_{ij}, \quad \forall (i, j) \in A, \\
& x_{ij}^k \geq 0, \quad \forall (i, j) \in A, k \in K.
\end{align*}

Los valores del parámetro $b_i^k$ se definen por medio de:
$$
b_i^k = \left\{
\begin{array}{rl}
-d_k, & \mbox{ si $i=s_k$,}\\
d_k, & \mbox{ si $i=t_k$,}\\
0, & \mbox{ en los demás casos.}
\end{array}
\right.
$$

Vamos a implementar este modelo usando la interfaz Python de Gurobi.



Definimos primero los conjuntos y parámetros del modelo:

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# Productos y nodos del grafo
K, s, t, d = gp.multidict({1: (2, 5, 10),
                        2: (1, 8, 15),
                        3: (3, 7, 12)})
V = gp.tuplelist(range(1,9))

# Arcos, capacidades y costos
A, u, c = gp.multidict({
  (2, 1):  (10, 1),
  (3, 4):  (20, 1),
  (6, 5):  (10, 1),
  (8, 7):  (12, 1),
  (2, 4):  (10, 1),
  (4, 6):  (30, 2),
  (6, 8):  (20, 1),
  (1, 3):  (25, 2),
  (3, 5):  (12, 1),
  (5, 7):  (12, 3),
  (1, 4):  (15, 1),
  (4, 5):  (25, 1),
  (5, 8):  (26, 1)})

# --- Desde aqui los valores se calculan en función de los anteriores
# Demandas asociadas a nodos y productos
b= {}
for i in V:
    for k in K:
        b[i,k] = -d[k] if i==s[k] else (d[k] if i==t[k] else 0) 


Creamos ahora el objeto modelo y las variables $x_{ij}^k$ del flujo multiproducto. Observar que las variables están indexadas por los conjuntos de productos y de arcos.

Si no se especifica el argumento `vtype` en la llamada a la función `addVars`, por defecto las variables creadas son continuas y no negativas.


In [ ]:
# Crear el objeto modelo
m = gp.Model('flujo-multiproducto')

# Crear las variables
x = m.addVars(A, K, name="x")

Definimos la función objetivo

In [ ]:
# Crear la funcion objetivo
m.setObjective(gp.quicksum(c[i,j]*x[i,j,k] 
                        for (i,j) in A for k in K), 
               GRB.MINIMIZE)

Definimos las restricciones del modelo:

In [ ]:
# Restricciones de demanda en los nodos
m.addConstrs(
    (x.sum('*',i,k) - x.sum(i,'*', k)  == b[i,k]
    for k in K for i in V), "demanda")

# Restricciones de capacidades en los arcos
m.addConstrs(
    (x.sum(i,j,'*') <= u[i,j] for i,j in A), "capacidad")

A menudo es útil exportar el modelo a un archivo de texto para poder revisarlo. Esto puede hacerse con el método `write`.

In [ ]:
# Escribir el modelo a un archivo
m.write('flujo_multiproducto.lp')

Finalmente, resolvemos el modelo y mostramos la solución. Notar que los valores de las variables pueden recuperarse también empleando el método `getAttr`:

In [ ]:
# Calcular la solucion optima
m.optimize()

# Escribir la solucion
if m.status == GRB.Status.OPTIMAL:
    # Recuperar los valores de las variables
    vx = m.getAttr('x', x)
    for k in K:
        print('\nFlujos optimos para {}:'.format(k))
        for i,j in A:
            if vx[i,j,k] > 0:
                print('{} -> {}: {}'.format(i, j, int(vx[i,j,k])))

## Código completo

Se reproduce a continuación el código completo del modelo anterior.

In [ ]:
# Implementación de modelos de programación lineal entera
# Problema de flujo multiproducto de costo mínimo (mincost flow)

# Luis M. Torres (EPN 2022)

import gurobipy as gp
from gurobipy import GRB

# Productos y nodos del grafo
K, s, t, d = gp.multidict({1: (2, 5, 10),
                        2: (1, 8, 15),
                        3: (3, 7, 12)})
V = gp.tuplelist(range(1,9))

# Arcos, capacidades y costos
A, u, c = gp.multidict({
  (2, 1):  (10, 1),
  (3, 4):  (20, 1),
  (6, 5):  (10, 1),
  (8, 7):  (12, 1),
  (2, 4):  (10, 1),
  (4, 6):  (30, 2),
  (6, 8):  (20, 1),
  (1, 3):  (25, 2),
  (3, 5):  (12, 1),
  (5, 7):  (12, 3),
  (1, 4):  (15, 1),
  (4, 5):  (25, 1),
  (5, 8):  (26, 1)})

# --- Desde aquí los valores se calculan en función de los anteriores
# Demandas asociadas a nodos y productos
b= {}
for i in V:
    for k in K:
        b[i,k] = -d[k] if i==s[k] else (d[k] if i==t[k] else 0) 

try:
    # Crear el objeto modelo
    m = gp.Model('flujo-multiproducto')

    # Crear las variables
    x = m.addVars(A, K, name="x")

    # Crear la función objetivo
    m.setObjective(gp.quicksum(c[i,j]*x[i,j,k] for (i,j) in A for k in K), GRB.MINIMIZE)

    # Restricciones de demanda en los nodos
    m.addConstrs(
        (x.sum('*',i,k) - x.sum(i,'*', k)  == b[i,k]
        for k in K for i in V), "demanda")

    # Restricciones de capacidades en los arcos
    m.addConstrs((x.sum(i,j,'*') <= u[i,j] for i,j in A), "capacidad")

    # Escribir el modelo a un archivo
    m.write('flujo_multiproducto.lp')

    # Resolver el modelo
    m.optimize()

    # Escribir la solución
    if m.status == GRB.Status.OPTIMAL:
        # Recuperar los valores de las variables
        vx = m.getAttr('x', x)
        for k in K:
            print('\nFlujos optimos para {}:'.format(k))
            for i,j in A:
                if vx[i,j,k] > 0:
                    print('{} -> {}: {}'.format(i, j, int(vx[i,j,k])))
                
except GurobiError as e:
    print('Se produjo un error de Gurobi: codigo: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error de atributo')

## Flujo multiproducto no divisible

Suponer que el transporte del producto $k$ desde el nodo $s_k$ hasta el nodo $t_k$ debe realizarse a lo largo de un único camino. ¿Cómo cambia el modelo?

In [ ]:
# Agregar variables binarias de seleccion de arcos
y = m.addVars(A, K, name="y", vtype = GRB.BINARY)

# Agregar restricciones de grado
m.addConstrs((y.sum('*',i,k) <= 1 for i in V for k in K), "entrante")
m.addConstrs((y.sum(i,'*',k) <= 1 for i in V for k in K), "saliente")

# Agregar restricciones de uso de arcos 
m.addConstrs((x[i,j,k] <= u[i,j]*y[i,j,k] for i,j in A for k in K), "uso")

# Actualizar el modelo
m.update()

# Escribir el modelo modificado a un archivo
m.write('flujo_no_divisible.lp')

# Resolver nuevamente
m.optimize()

# Mostrar la solucion
if m.status == GRB.Status.OPTIMAL:
    vx = m.getAttr('x', x)
    for k in K:
        print('\nFlujos optimos para {}:'.format(k))
        for i,j in A:
            if vx[i,j,k] > 0:
                print('{} -> {}: {}'.format(i, j, vx[i,j,k]))
